In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
import chemprop

In [2]:
import sys
sys.path.append('../../../../code')

from metrics import get_hi_metrics
from utils import chemprop_prepare_df, chemprop_process_folder

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
train = pd.read_csv('../../../../data/hi/drd2/train_1.csv')
test = pd.read_csv('../../../../data/hi/drd2/test_1.csv')

train

,Unnamed: 0,smiles,value
0,383,CC(C)Oc1ccccc1N1CCN(Cc2cccc(C(=O)N3CCCCC3)c2)CC1,True
1,386,CC(C)Oc1ccccc1N1CCN(Cc2cccc(CN3CCCCC3=O)c2)CC1,True
2,389,CC(C)Oc1ccccc1N1CCN(Cc2ccccc2CN2CCCCC2=O)CC1,True
3,2695,COc1ccccc1N1CCN(CC2COCC(c3ccccc3)(c3ccccc3)O2)CC1,True
4,2995,COc1ccccc1N1CCN(C[C@H]2OCCOC2(c2ccccc2)c2ccccc...,False
...,...,...,...
2380,5444,O=C1c2ccccc2C(=O)N1CCCCN1CCCN(C(c2ccccc2)c2ccc...,True
2381,4391,O=C(CCC(=O)c1ccccc1)NCCc1c[nH]c2ccccc12,False
2382,4397,O=C(CCCC(=O)c1ccccc1)NCCc1c[nH]c2ccccc12,False
2383,5999,OC12C3C4CC5C6C4C1C6C(C53)N2CC1CCCCC1,False


In [4]:
import os
import os
import shutil

temp_datapath = '/tmp/chemprop/'
if os.path.exists(temp_datapath) and os.path.isdir(temp_datapath):
    shutil.rmtree(temp_datapath)
os.mkdir(temp_datapath)

input_paths = [
    '/data/steshin/gero_benchmark/data/hi/drd2/'
]

output_paths = [
    temp_datapath
]

for i in range(len(input_paths)):
    chemprop_process_folder(input_paths[i], output_paths[i])

In [5]:
base_arguments = [
    '--dataset_type', 'classification', 
    '--data_path', temp_datapath + 'train_1.csv',
    '--separate_val_path', temp_datapath + 'test_1.csv',
    '--separate_test_path', temp_datapath + 'test_1.csv',
    '--save_preds',
    '--metric', 'prc-auc',
    '--epochs', '100',
    '--gpu', '1',
    '--save_dir', temp_datapath + 'checkpoint',
    '--features_generator', 'rdkit_2d_normalized',
    '--no_features_scaling',
]

In [6]:
param_dict = {
    '--depth': ['3', '4', '5', '6'],
    '--dropout': ['0.0', '0.2', '0.3', '0.5', '0.7'],
    '--ffn_hidden_size': ['600', '1200', '2400', '3600'],
    '--ffn_num_layers': ['1', '2', '3'],
    '--hidden_size': ['600', '1200', '2400', '3600']
}

In [7]:
from copy import copy
from tqdm import tqdm
from sklearn.model_selection import ParameterSampler

hyperparam_list = list(ParameterSampler(param_dict, n_iter=20))

best_score = 0.0
best_hyperparams = None

for i, hyperparam in tqdm(enumerate(hyperparam_list)):
    print('HYPERPARAMETER', i)
    print(hyperparam)
    result_arguments = copy(base_arguments)
    for key, value in hyperparam.items():
        result_arguments.append(key)
        result_arguments.append(value)
    # Train
    args = chemprop.args.TrainArgs().parse_args(result_arguments)
    mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

    test_preds = pd.read_csv(temp_datapath + 'checkpoint/test_preds.csv')
    metrics = get_hi_metrics(test, test_preds['targets'].to_list())

    score = metrics['prc_auc']
    if score > best_score:
        print('New best PRC AUC:', score)
        best_score = score
        best_hyperparams = hyperparam

0it [00:00, ?it/s]Command line
python /home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/ipykernel_launcher.py --ip=127.0.0.1 --stdin=9043 --control=9041 --hb=9040 --Session.signature_scheme="hmac-sha256" --Session.key=b"833b9029-1941-4e7a-a21a-001d99371c30" --shell=9042 --transport="tcp" --iopub=9044 --f=/home/steshin/.local/share/jupyter/runtime/kernel-v2-2428590KJktLDJJkrJ.json
Args
{'activation': 'ReLU',
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_feature_scaling': True,
 'bond_features_path': None,
 'bond_features_size': 0,
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_path': None,
 '

HYPERPARAMETER 0
{'--hidden_size': '600', '--ffn_num_layers': '1', '--ffn_hidden_size': '2400', '--dropout': '0.5', '--depth': '3'}


2385it [00:00, 383228.56it/s]
100%|██████████| 2385/2385 [00:00<00:00, 165190.07it/s]
Number of tasks = 1
Fold 0
Splitting data with seed 0
1190it [00:00, 400804.77it/s]
100%|██████████| 1190/1190 [00:00<00:00, 171820.78it/s]
1190it [00:00, 420809.52it/s]
100%|██████████| 1190/1190 [00:00<00:00, 168128.20it/s]
Class sizes
targets 0: 29.39%, 1: 70.61%
Total size = 2,385 | train size = 2,385 | val size = 1,190 | test size = 1,190
Building model 0
MoleculeModel(
  (sigmoid): Sigmoid()
  (encoder): MPN(
    (encoder): ModuleList(
      (0): MPNEncoder(
        (dropout_layer): Dropout(p=0.5, inplace=False)
        (act_func): ReLU()
        (W_i): Linear(in_features=147, out_features=600, bias=False)
        (W_h): Linear(in_features=600, out_features=600, bias=False)
        (W_o): Linear(in_features=733, out_features=600, bias=True)
      )
    )
  )
  (ffn): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=800, out_features=1, bias=True)
  )
)
Number of par

New best PRC AUC: 0.740729523267746
HYPERPARAMETER 1
{'--hidden_size': '1200', '--ffn_num_layers': '2', '--ffn_hidden_size': '1200', '--dropout': '0.7', '--depth': '5'}


2385it [00:00, 360665.38it/s]
100%|██████████| 2385/2385 [00:00<00:00, 161846.61it/s]
Number of tasks = 1
Fold 0
Splitting data with seed 0
1190it [00:00, 344483.52it/s]
100%|██████████| 1190/1190 [00:00<00:00, 172952.00it/s]
1190it [00:00, 369091.31it/s]
100%|██████████| 1190/1190 [00:00<00:00, 158637.82it/s]
Class sizes
targets 0: 29.39%, 1: 70.61%
Total size = 2,385 | train size = 2,385 | val size = 1,190 | test size = 1,190
Building model 0
MoleculeModel(
  (sigmoid): Sigmoid()
  (encoder): MPN(
    (encoder): ModuleList(
      (0): MPNEncoder(
        (dropout_layer): Dropout(p=0.7, inplace=False)
        (act_func): ReLU()
        (W_i): Linear(in_features=147, out_features=1200, bias=False)
        (W_h): Linear(in_features=1200, out_features=1200, bias=False)
        (W_o): Linear(in_features=1333, out_features=1200, bias=True)
      )
    )
  )
  (ffn): Sequential(
    (0): Dropout(p=0.7, inplace=False)
    (1): Linear(in_features=1400, out_features=1200, bias=True)
    (2): R

...

2385it [00:00, 374393.32it/s]
100%|██████████| 2385/2385 [00:00<00:00, 156489.19it/s]
Number of tasks = 1
Fold 0
Splitting data with seed 0
1190it [00:00, 458457.04it/s]
100%|██████████| 1190/1190 [00:00<00:00, 169004.90it/s]
1190it [00:00, 417012.43it/s]
100%|██████████| 1190/1190 [00:00<00:00, 169274.29it/s]
Class sizes
targets 0: 29.39%, 1: 70.61%
Total size = 2,385 | train size = 2,385 | val size = 1,190 | test size = 1,190
Building model 0
MoleculeModel(
  (sigmoid): Sigmoid()
  (encoder): MPN(
    (encoder): ModuleList(
      (0): MPNEncoder(
        (dropout_layer): Dropout(p=0.2, inplace=False)
        (act_func): ReLU()
        (W_i): Linear(in_features=147, out_features=2400, bias=False)
        (W_h): Linear(in_features=2400, out_features=2400, bias=False)
        (W_o): Linear(in_features=2533, out_features=2400, bias=True)
      )
    )
  )
  (ffn): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=2600, out_features=600, bias=True)
    (2): Re

HYPERPARAMETER 18
{'--hidden_size': '2400', '--ffn_num_layers': '2', '--ffn_hidden_size': '600', '--dropout': '0.7', '--depth': '4'}


2385it [00:00, 357865.53it/s]
100%|██████████| 2385/2385 [00:00<00:00, 164364.95it/s]
Number of tasks = 1
Fold 0
Splitting data with seed 0
1190it [00:00, 339331.14it/s]
100%|██████████| 1190/1190 [00:00<00:00, 161632.83it/s]
1190it [00:00, 240749.65it/s]
100%|██████████| 1190/1190 [00:00<00:00, 164255.17it/s]
Class sizes
targets 0: 29.39%, 1: 70.61%
Total size = 2,385 | train size = 2,385 | val size = 1,190 | test size = 1,190
Building model 0
MoleculeModel(
  (sigmoid): Sigmoid()
  (encoder): MPN(
    (encoder): ModuleList(
      (0): MPNEncoder(
        (dropout_layer): Dropout(p=0.7, inplace=False)
        (act_func): ReLU()
        (W_i): Linear(in_features=147, out_features=2400, bias=False)
        (W_h): Linear(in_features=2400, out_features=2400, bias=False)
        (W_o): Linear(in_features=2533, out_features=2400, bias=True)
      )
    )
  )
  (ffn): Sequential(
    (0): Dropout(p=0.7, inplace=False)
    (1): Linear(in_features=2600, out_features=600, bias=True)
    (2): Re

HYPERPARAMETER 19
{'--hidden_size': '2400', '--ffn_num_layers': '3', '--ffn_hidden_size': '2400', '--dropout': '0.3', '--depth': '6'}


2385it [00:00, 375672.79it/s]
100%|██████████| 2385/2385 [00:00<00:00, 158119.26it/s]
Number of tasks = 1
Fold 0
Splitting data with seed 0
1190it [00:00, 451122.72it/s]
100%|██████████| 1190/1190 [00:00<00:00, 165469.49it/s]
1190it [00:00, 413865.82it/s]
100%|██████████| 1190/1190 [00:00<00:00, 165442.07it/s]
Class sizes
targets 0: 29.39%, 1: 70.61%
Total size = 2,385 | train size = 2,385 | val size = 1,190 | test size = 1,190
Building model 0
MoleculeModel(
  (sigmoid): Sigmoid()
  (encoder): MPN(
    (encoder): ModuleList(
      (0): MPNEncoder(
        (dropout_layer): Dropout(p=0.3, inplace=False)
        (act_func): ReLU()
        (W_i): Linear(in_features=147, out_features=2400, bias=False)
        (W_h): Linear(in_features=2400, out_features=2400, bias=False)
        (W_o): Linear(in_features=2533, out_features=2400, bias=True)
      )
    )
  )
  (ffn): Sequential(
    (0): Dropout(p=0.3, inplace=False)
    (1): Linear(in_features=2600, out_features=2400, bias=True)
    (2): R

In [8]:
print(best_hyperparams)

{'--hidden_size': '2400', '--ffn_num_layers': '1', '--ffn_hidden_size': '2400', '--dropout': '0.0', '--depth': '6'}


In [9]:
print(best_score)

0.7582185136305537


In [10]:
from copy import copy

result_arguments = copy(base_arguments)
result_arguments.extend([
    '--hidden_size', '2400',
    '--ffn_num_layers', '1',
    '--ffn_hidden_size', '2400',
    '--dropout', '0.0',
    '--depth', '6'
])

args = chemprop.args.TrainArgs().parse_args(result_arguments)
mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)


Command line
python /home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/ipykernel_launcher.py --ip=127.0.0.1 --stdin=9043 --control=9041 --hb=9040 --Session.signature_scheme="hmac-sha256" --Session.key=b"833b9029-1941-4e7a-a21a-001d99371c30" --shell=9042 --transport="tcp" --iopub=9044 --f=/home/steshin/.local/share/jupyter/runtime/kernel-v2-2428590KJktLDJJkrJ.json
Args
{'activation': 'ReLU',
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_feature_scaling': True,
 'bond_features_path': None,
 'bond_features_size': 0,
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_path': None,
 'crossval_index_dir

In [9]:
max_epoch = 6